In [ ]:
import importlib
import pandas as pd
import numpy as np
import re

from functions.functions_dataframe import * 
from parameters.filters_lists import *
from parameters.regs import *

pd.set_option("display.max_colwidth", 200)

In [ ]:
##### Step 1. Завантажити датафрейми ProZorro, МОН, створити список ЄДРПОУ освітніх закладів
""" Завантажити базу закупівель ProZorro """
procurements = pd.read_csv('data/2019-2020.csv', sep = ';', low_memory = False)
""" Завантажити датафрейм МОН навчальниз закладів """
mon_data_df = pd.read_csv('data/mon_data/MON_schools_dnz.csv', sep = ';')
""" Створити список унікальних ЄДРПОУ освітніх закладів МОН """
edrpou_unique_list = mon_data_df.edrpou.unique().tolist()

In [ ]:
## Опрацювання шкіл замовників

### Step 2: Створити базу навчальних закладів, які самостійно купують через ProZorro
#### Відфільтрувати базу закупівель procurements за колонкою IDOrganizator.
#### Фільтром є список МОН з ЄДРПОУ начальних закладів edrpou_unique_list
#### tenders_by_schools_raw - база закупівель, де навчальні заклади є замовниками
#### procurements_undefined_1 - база невизначених закупівель ProZorro
tenders_by_schools_raw = procurements[procurements['IDOrganizator'].isin(edrpou_unique_list)]
tenders_by_schools_undefined_1 = procurements[~procurements['IDOrganizator'].isin(edrpou_unique_list)]
tenders_by_schools_undefined_2 = tenders_by_schools_raw[tenders_by_schools_raw['Организатор'].str.contains('управління освіти', case=False)]
tenders_by_schools_raw = tenders_by_schools_raw[~tenders_by_schools_raw['Организатор'].str.contains('управління освіти', case=False)]

### Step 3: Класифікація шкіл-замовників по типах за колонкою "Организатор"
tenders_by_schools_classified = classification_by_type_one_column(tenders_by_schools_raw, ['Организатор'], filter_priority, filter_name_priority, 'Тип закладу')

### Step 4: Нумерація шкіл-замовників
tenders_by_schools_numerated = numeration_one_column(tenders_by_schools_classified, ['Организатор'], reg_number)

### Step 5: Cтворення фінальної бази шкіл замовників
tenders_by_schools = tenders_by_schools_numerated.copy()
tenders_by_schools['Номер'] = tenders_by_schools['Номер'].replace(r'^\s*$', np.nan, regex=True)
tenders_by_schools['Назва закладу'] = np.where(tenders_by_schools['Номер'].notnull(), tenders_by_schools['Тип закладу'] + ' ' + tenders_by_schools['Номер'], tenders_by_schools['Организатор'])

#### Створення колонки 'Заклад-замовник'. Видалення технічних колонок, створених функціями.
tenders_by_schools['Заклад-замовник'] = True
col_names = procurements.columns.tolist()
col_names += ['Заклад-замовник', 'Назва закладу']
tenders_by_schools = tenders_by_schools[col_names]

In [ ]:
## ШКОЛИ НЕ-ЗАМОВНИКИ

### Step 5: Пошук закупівель шкіл-не-замовників за допомогою ключових слів
### З-поміж датафрейму невизначених закупівель (procurements_undefined_1) за ключовими словами у списках
### filter_keywords, filter_keywords_strict відбираються закупівлі освітніх закладів у колонках: Тендер, ОписаниеТендера, Лот 
### регулярний вираз /b{}/b шукає за ключовими словами зі списку filter_keywords_strict_extended тільки ті паттерни, де ці ключові слова є окремими, а не зустрічають всередині слів
procurements_undefined_1 = pd.concat([tenders_by_schools_undefined_1, tenders_by_schools_undefined_2])
tenders_not_by_schools_by_keywords = procurements_undefined_1[procurements_undefined_1['Тендер'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True)  | 
                                                      procurements_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | 
                                                      procurements_undefined_1['Лот'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | procurements_undefined_1['Тендер'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | procurements_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | procurements_undefined_1['Лот'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True)]

### Step 6: Класифікація шкіл-не-замовників за типами
tenders_not_by_schools_classified = classification_by_type_three_cols(tenders_not_by_schools_by_keywords,
                                                                         ['Тендер', 'ОписаниеТендера', 'Лот'], 
                                                                         filter_priority, 
                                                                         filter_name_priority)

### Step 7: Нумерація шкіл-не-замовників
#### df_step7 - датафрейм, створений із закупівель tenders_not_by_schools_classified, де був успішно визначений "Тип закладу"
#### long_numeration - функція, яка створює два дафайреми:
####    1) tenders_not_by_schools_numerated - тендери, де номер визначився
####    2) step7_numerated_undefined - тендери, де номер не визначився
df_step7 = tenders_not_by_schools_classified[tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()
tenders_not_by_schools_numerated, step7_numerated_undefined = long_numeration(df_step7, reg_number)
tenders_not_by_schools_numerated['Назва закладу'] = tenders_not_by_schools_numerated['Тип закладу'] + ' ' + tenders_not_by_schools_numerated['Номер']

### Step 8: Найменування шкіл-не-замовників
#### df_names_clean - власні назви навчальниз закладів від МОН
#### school_names - список унікальних назв 
#### long_naming - функція, яка створює два дафайреми:
####    1) tenders_not_by_schools_named - тендери, де номер визначився.
####    2) tenders_not_by_schools_undefined - тендери шкіл не замовників, де не визначилась назва і номер. Такі закупівлі вважаються закупівлями управінь освіти для різних закладів.

df_names_clean = pd.read_csv('data/mon_data/MON_names.csv', sep = ';')
df_names_clean = df_names_clean[df_names_clean['Назва'].notnull()] # Видалити NaN
school_names = df_names_clean['Назва'].unique().tolist()
tenders_not_by_schools_named, tenders_not_by_schools_undefined = long_naming(step7_numerated_undefined, reg_double_quotes, school_names)
tenders_not_by_schools_named['Назва закладу'] = tenders_not_by_schools_named['Тип закладу'] + ' ' + tenders_not_by_schools_named['Назва']

### Step 9: Об'єднання двох датафреймів шкіл-не-замовників
#### tenders_not_by_schools - датафрейм шкіл-не-замовників з визначеними номерами і назвами
frames = [tenders_not_by_schools_numerated, tenders_not_by_schools_named]
tenders_not_by_schools = pd.concat(frames)
tenders_not_by_schools['Заклад-замовник'] = False
tenders_not_by_schools = tenders_not_by_schools[col_names]

In [ ]:
### Для дебагу
### Треба доробити фільтрацію для шкіл замовників (tenders_by_schools), бо чимало відділів освіти сюди потрапило
tenders_by_schools['Организатор'].value_counts()

In [ ]:
### Зберегти результати окремо для шкіл замовинків і не замовників
# tenders_by_schools.to_csv('results/tenders_by_schools.csv', sep=';', index=False)
# tenders_not_by_schools.to_csv('results/tenders_not_by_schools.csv', sep=';', index=False)

In [ ]:
## ВСІ ШКОЛИ

##### Step 10: Об'єднання датафреймів шкіл-не-замовників та шкіл-замовників
tenders_concatenated = pd.concat([tenders_by_schools, tenders_not_by_schools])
tenders_concatenated['Назва закладу'] = tenders_concatenated['Назва закладу'].str.replace('№ ', '№')
tenders_concatenated.to_csv('results/2020_result.csv', sep=';', index=False)